## Poročilo vaja 3.3
#### Preverjanje hipotez po članku: Anketa za nogometne trenerje

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy.stats import mannwhitneyu

In [2]:
pd.set_option('display.max_rows', None)

In [6]:
path = os.getcwd()
df = pd.read_excel(path + '\\data\\anketa114621-2024-03-04.xlsx')
df.head()

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,Q39,Q40,Q40_3_text,Q41,Q42,Q42_13_text,Q43,Q43_4_text,Q44,Q44_3_text
0,Koliko ur na teden delate kot trener (s svojo ...,Koliko let že delate kot trener?,Koliko let že delate kot trener v trenutnem kl...,Kakšna je vaša trenerska usposboljenost?,Kakšne je vaša trenutna trenerska licenca (sez...,Koliko dni na teden ste na nogometnem igrišču ...,Koliko ur na teden preživite v nogometnem klub...,Koliko ur od tega je namenjeno na teden za pr...,Koliko ur na teden delate doma za svojo ekipo ...,Ocenite koliko ur na teden porabite za pridobi...,...,Član katerega društva nogemetnih trenerjev ste?,Vaš spol,Drugo:,Kaj je vaša najvišja stopnja izobrazbe?,Na katerem področju ste pridobili to diplomo? ...,Drugo:,Kakšen je vaš zaposlitveni status?,Drugo:,Ali ste redno zaposleni?,Drugo:
1,9,15,15,2,2,4,20,10,3,1,...,5,2,-2,4,3,-2,3,-2,2,-2
2,12,10,3,4,2,6,15,12,5,1,...,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3
3,6,16,8,2,2,5,11,7,5,4,...,5,2,-2,2,-1,-1,2,-2,2,-2
4,6,3,3,1,1,4,8,6,2,1,...,5,2,-2,4,13,Strojništvo,1,-2,3,-1


In [7]:
# assign the 0th row to questions variable and remove it
questions = df.iloc[0]
df.drop([0], inplace=True)

In [29]:
# create binomial variables: education (high-school or lower/ higher than high-school)
#                            field (sports / other)
df['education'] = np.where(df['Q41'].isin([1, 2]), 'srednja šola ali manj', np.where(df['Q41'].isin([3, 4]), 'višješolska ali več', pd.NA))
df['field'] = np.where(df['Q42'].isin([11]), 'športna izobrazba', np.where(df['Q42'] > 0, 'drugo', pd.NA))

df_education = df.dropna(subset=['education']).copy()
df_field = df.dropna(subset=['field']).copy()

**Metodologija:** V analizi primerjamo ali obstajajo razlike med skupinama v mnenju, kateri dejavniki so pomembni za trenerski poklic, kako ocenjujejo zadovoljstvo in kaj je glavni problem pri delu. Anketiranci so na zastavljena vprašanja odgovarjali s pomočjo odgovorov 5 stopenjske Likertove lestvice. Takšni podatki so ordinalni, zato za analizo uporabimo neparametričen test; *Mann-Withney U* test. 

In [18]:

def mann_whitney_results(data, variable, var_condition, filt):
    """
    Calculates the Mann-Whitney U test on non-normally distributed data 
    :param data: pd.DataFrame containing the data
    :param variable: group by the variable
    :param var_condition: group condition
    :param filt: by what substring to filter subquestion labels. E.g. Q22a, Q22b --> filter = Q22
    :return: returns pd.DataFrame of the results
    """
    
    results = []
    
    data_filtered = data.filter(regex=filt)
    
    columns = data_filtered.columns
    
    
    for c in columns:
        gr_less = data[data[variable] == var_condition]
        gr_more = data[data[variable] != var_condition]
        
        gr_less = gr_less[gr_less[c] >= 1][c].astype(np.int64)
        gr_more = gr_more[gr_more[c] >= 1][c].astype(np.int64)
        
        stat, p = mannwhitneyu(gr_less, gr_more)
        results.append({
            'Statement': f'{c}: {questions[c]}',
            'U statistic': round(stat, 2),
            'p-value': round(p, 2)
        })
    return pd.DataFrame(results)


In [45]:
q22_table_ed = mann_whitney_results(data=df_education, variable='education', var_condition='srednja šola ali manj', filt='Q22')

display(result_table.style.set_caption('Q22: Kako pomembne so z ozirom na Vašo predstavo o trenerskem poklicu naslednji dejavniki? Razlika v stopnji izobrazbe')\
          .set_table_styles([
              {'selector': 'caption', 'props': [('font-size', '16px'), 
                                                ('font-weight', 'bold'), 
                                                ('border-bottom', '1px solid black'),
                                                ('width', '100%')]},
              {'selector': 'td', 'props': 'text-align: left;'}
          ])\
          .hide())

Statement,U statistic,p-value
Q22a: Dobri medsebojni odnosi z igralci/igralkami,23913.000000,0.220000
Q22b: Dobri medsebojni odnosi med trenerji,24704.000000,0.070000
Q22c: Dobri medsebojni odnosi z vodstvom kluba,23350.000000,0.540000
Q22d: Izpolnjevanje vizije kluba,23362.500000,0.540000
Q22e: Finančna stabilnost kluba,23197.000000,0.630000
Q22f: Organiziranost kluba,22697.000000,0.890000
Q22g: Možnost napredovanja,23118.000000,0.740000
Q22h: Strokovna usposobljenost,20240.000000,0.030000
Q22i: Časovna fleksibilnost,22903.000000,0.830000
Q22j: Predhodne izkušnje in dosežki,24329.500000,0.190000


**Interpretacija:** Iz rezultatov Mann-Whitney U testa lahko vidimo, da se odgovor skupine z višjo in skupine z nižjo izobrazbo statistično značilno razlikujejo v primerih trditve ***Q22g, Q22k, in Q22t***. To pomeni, da se mnenja o možnosti napredovanja, prepoznavnosti v družbi in odnosu z navijači kot pomembnemu dejavniku za predstavo o trenerskem poklicu med skupinama statistično značilno razlikujejo. 

In [44]:
q22_table_fi = mann_whitney_results(data=df_field, variable='field', var_condition='športna izobrazba', filt='Q22')

display(q22_table_fi.style.set_caption('Q22: Kako pomembne so z ozirom na Vašo predstavo o trenerskem poklicu naslednji dejavniki? Razlika v področju izobrazbe')\
          .set_table_styles([
              {'selector': 'caption', 'props': [('font-size', '16px'), 
                                                ('font-weight', 'bold'), 
                                                ('border-bottom', '1px solid black'),
                                                ('width', '100%')]},
              {'selector': 'td', 'props': 'text-align: left;'}
          ])\
          .hide())

Statement,U statistic,p-value
Q22a: Dobri medsebojni odnosi z igralci/igralkami,23913.000000,0.220000
Q22b: Dobri medsebojni odnosi med trenerji,24704.000000,0.070000
Q22c: Dobri medsebojni odnosi z vodstvom kluba,23350.000000,0.540000
Q22d: Izpolnjevanje vizije kluba,23362.500000,0.540000
Q22e: Finančna stabilnost kluba,23197.000000,0.630000
Q22f: Organiziranost kluba,22697.000000,0.890000
Q22g: Možnost napredovanja,23118.000000,0.740000
Q22h: Strokovna usposobljenost,20240.000000,0.030000
Q22i: Časovna fleksibilnost,22903.000000,0.830000
Q22j: Predhodne izkušnje in dosežki,24329.500000,0.190000


**Interpretacija:** Iz rezultatov Mann-Whitney U testa lahko vidimo, da se odgovori skupine s športno in skupine ne športno izobrazbo statistično značilno razlikujejo v le primeru trditve ***Q22h***. To pomeni, da se mnenje o strokovni usposobljenosti kot pomembnemu dejavniku za predstavo o trenerskem poklicu med skupinama statistično značilno razlikujejo. 

In [46]:
q23_table = mann_whitney_results(data=df_education, variable='education', var_condition='srednja šola ali manj', filt='Q23')
q34_table = mann_whitney_results(data=df_education, variable='education', var_condition='srednja šola ali manj', filt='Q34')

display(q23_table.style.set_caption('Q23: Biti zadovoljen s trenerskim poklicem je odvisno od več stvari. Kako ocenjujete naslednje vidike in pogoje vaše poklicne usmeritve? Razlika v stopnji izobrazbe')\
          .set_table_styles([
              {'selector': 'caption', 'props': [('font-size', '16px'), 
                                                ('font-weight', 'bold'), 
                                                ('border-bottom', '1px solid black'),
                                                ('width', '100%')]},
              {'selector': 'td', 'props': 'text-align: left;'}
          ])\
          .hide())
display(q34_table.style.set_caption('Q34: Ocenite dejavnike, ki so za vas pomembni za zadovoljstvo pri delu. Razlika v stopnji izobrazbe')\
          .set_table_styles([
              {'selector': 'caption', 'props': [('font-size', '16px'), 
                                                ('font-weight', 'bold'), 
                                                ('border-bottom', '1px solid black'),
                                                ('width', '100%')]},
              {'selector': 'td', 'props': 'text-align: left;'}
          ])\
          .hide())

Statement,U statistic,p-value
Q23a: uresničevanje svojih idej v praksi,32595.000000,0.930000
Q23b: možnost raznovrstnega treniranja,31360.000000,0.340000
Q23c: pogoji za trening in tekmovanje,33759.000000,0.450000
Q23d: stiki z drugimi strokovnjaki,31558.000000,0.420000
Q23e: ustvarjalno delo,32185.500000,0.710000
Q23f: rezultatski uspehi,35278.000000,0.110000
Q23g: napredek mladih nogometašev,32951.000000,0.860000
Q23h: podpora vodstvenih struktur v klubu,32312.000000,0.710000
Q23i: Podpora vodilnih na NZS,36857.500000,0.010000


Statement,U statistic,p-value
Q34a: Pogoji dela,34127.000000,0.250000
Q34b: Redno plačilo za opravljeno delo,31578.500000,0.610000
Q34c: Možnosti napredovanja,32869.000000,0.650000
Q34d: Nenehno dokazovanje,37096.500000,0.000000
Q34e: Količina stresa pri delu,30524.000000,0.310000
Q34f: Spoštovanje,33702.500000,0.400000
"Q34g: Priznavanje znanja, izkušenj",33638.500000,0.430000
Q34h: Vrednotenje rezultatov dela,34119.500000,0.230000
Q34i: Podpora vodilnih v klubu,33529.000000,0.410000
Q34j: Dolgoročnost pogodbe o zaposlitvi,33307.000000,0.260000


**Interpretacija:** Iz podatkov lahko sklepamo, da imata skupini statistično značilno različno mnenje glede podpore vodilnih na NZS (***Q23i***) in nenehnega dokazovanja (***Q34d***) kot dejavnik ocene zadovoljstva pri delu.

In [47]:
q23_table = mann_whitney_results(data=df_clean, variable='field', var_condition='športna izobrazba', filt='Q23')
q34_table = mann_whitney_results(data=df_clean, variable='field', var_condition='športna izobrazba', filt='Q34')

display(q23_table.style.set_caption('Q23: Biti zadovoljen s trenerskim poklicem je odvisno od več stvari. Kako ocenjujete naslednje vidike in pogoje vaše poklicne usmeritve? Razlika v področju izobrazbe')\
          .set_table_styles([
              {'selector': 'caption', 'props': [('font-size', '16px'), 
                                                ('font-weight', 'bold'), 
                                                ('border-bottom', '1px solid black'),
                                                ('width', '100%')]},
              {'selector': 'td', 'props': 'text-align: left;'}
          ])\
          .hide())
display(q34_table.style.set_caption('Q34: Ocenite dejavnike, ki so za vas pomembni za zadovoljstvo pri delu. Razlika v področju izobrazbe')\
          .set_table_styles([
              {'selector': 'caption', 'props': [('font-size', '16px'), 
                                                ('font-weight', 'bold'), 
                                                ('border-bottom', '1px solid black'),
                                                ('width', '100%')]},
              {'selector': 'td', 'props': 'text-align: left;'}
          ])\
          .hide())

Statement,U statistic,p-value
Q23a: uresničevanje svojih idej v praksi,20580.000000,0.130000
Q23b: možnost raznovrstnega treniranja,21994.000000,0.700000
Q23c: pogoji za trening in tekmovanje,22656.000000,0.790000
Q23d: stiki z drugimi strokovnjaki,21762.500000,0.600000
Q23e: ustvarjalno delo,20142.500000,0.050000
Q23f: rezultatski uspehi,22934.500000,0.680000
Q23g: napredek mladih nogometašev,20802.500000,0.120000
Q23h: podpora vodstvenih struktur v klubu,21983.000000,0.690000
Q23i: Podpora vodilnih na NZS,24495.500000,0.090000


Statement,U statistic,p-value
Q34a: Pogoji dela,23281.000000,0.310000
Q34b: Redno plačilo za opravljeno delo,22036.000000,0.990000
Q34c: Možnosti napredovanja,22023.500000,0.970000
Q34d: Nenehno dokazovanje,22761.000000,0.590000
Q34e: Količina stresa pri delu,20217.500000,0.140000
Q34f: Spoštovanje,22020.000000,0.930000
"Q34g: Priznavanje znanja, izkušenj",21632.000000,0.680000
Q34h: Vrednotenje rezultatov dela,21898.500000,0.890000
Q34i: Podpora vodilnih v klubu,22490.000000,0.700000
Q34j: Dolgoročnost pogodbe o zaposlitvi,22253.000000,0.500000


**Interpretacija**: Skupini z različnimi področji izobrazbe se mejno statistično značilno razlikujeta v odgovoru ***Q23e*** (ustvarjalnost pri delu), kot dejavniku pri oceni zadovoljstva pri delu. Glede na to, da je p-vrednost testa enaka mejni vrednosti smo se odločili za konzervativnejši pristop in tako poročamo, da med skupinama z različnimi področji izobrazbe ni statistično značilnih razlik v mnenju kateri dejavniki vplivajo na zadovoljstvo pri delu.

In [43]:
q32_table_ed = mann_whitney_results(data=df_clean, variable='education', var_condition='srednja šola ali manj', filt='Q32')
q32_table_fi = mann_whitney_results(data=df_clean, variable='field', var_condition='športna izobrazba', filt='Q32')

display(q32_table_ed.style.set_caption('Q32: Kje so glavni problemi pri delu v vašem klubu? Razlika v stopnji izobrazbe')\
          .set_table_styles([
              {'selector': 'caption', 'props': [('font-size', '16px'), 
                                                ('font-weight', 'bold'), 
                                                ('border-bottom', '1px solid black'),
                                                ('width', '100%')]},
              {'selector': 'td', 'props': 'text-align: left;'}
          ])\
          .hide())
display(q32_table_fi.style.set_caption('Q32: Kje so glavni problemi pri delu v vašem klubu? Razlika v področju izobrazbe')\
          .set_table_styles([
              {'selector': 'caption', 'props': [('font-size', '16px'), 
                                                ('font-weight', 'bold'), 
                                                ('border-bottom', '1px solid black'),
                                                ('width', '100%')]},
              {'selector': 'td', 'props': 'text-align: left;'}
          ])\
          .hide())

Statement,U statistic,p-value
Q32a: Slabi pogoji za treniranje,25833.000000,0.300000
Q32b: Neurejeni medčloveški odnosi,25040.000000,0.710000
Q32c: Premalo nogometašev je pripravljenih za delo,24059.000000,0.750000
Q32d: Slaba organizacija,25135.500000,0.610000
"Q32e: Neustrezno razporejena finančna sredstva, preveč v korist članskih moštev, premalo za mlajše kategorije",23350.500000,0.580000
Q32f: Slaba infrastruktura (nogometno igrišče),24845.500000,0.680000
Q32g: Nestrpnost pri doseganju ciljev,25677.000000,0.300000
Q32h: Zanemarjanje razvoja lastnih igralcev,26489.000000,0.140000


Statement,U statistic,p-value
Q32a: Slabi pogoji za treniranje,23995.500000,0.070000
Q32b: Neurejeni medčloveški odnosi,21557.000000,0.810000
Q32c: Premalo nogometašev je pripravljenih za delo,22548.500000,0.530000
Q32d: Slaba organizacija,21431.000000,0.780000
"Q32e: Neustrezno razporejena finančna sredstva, preveč v korist članskih moštev, premalo za mlajše kategorije",21398.000000,0.950000
Q32f: Slaba infrastruktura (nogometno igrišče),23256.000000,0.210000
Q32g: Nestrpnost pri doseganju ciljev,21506.000000,0.870000
Q32h: Zanemarjanje razvoja lastnih igralcev,21456.000000,0.760000


**Intepretacija:** Iz opravljenih statističnih testov je razvidno, da med skupinama ni razlik o mnenju, kaj so glavni problemi v klubu.